In [1]:
from ds_utils.ds_preamble import *
from ds_utils.ds_plotting import * 
from ds_utils.ds_helper import *
np.random.seed(0)
a = np.random.choice(15, size=20)
# print(a)

calling ds_preamble..
calling ds_helper...


In [ ]:
# -------------------------------- backup code ------------------------------- #
# further sanity check...
# import numpy as np
# for i in range(100):
#     res=[]
#     a = np.random.randint(100, size=i)
#     selection_sort(a)
#     res.append(all(a == sorted(a)))

# print(f"all checks passed? {all(res)}")

"""
def sanity_check_sorted(sort_func, verbose=False):
    for i in range(30):
        a = np.random.randint(100, size=i)
        # b = np.random.random(size=i)
        
        # print(f"sorting {a}:")
        sort_func(a)
        # print(f"sorted: {a}")

        # print(f"sorting {b}:")
        # sort_func(b)
        # print(f"sorted: {b}")

        # if not (is_sorted(a) and is_sorted(b)):
        if not is_sorted(a):
            print(f"failed test case: after sorting is {a}")
            return False
    return True
            

def sorting_decorator(func):
    def wrapper():
        print(f'calling "{func.__name__}": ')
        if sanity_check_sorted(func):
            print("checks passed")
        else:
            print("checks failed!")
    return wrapper


# same as "my_sort = sorting_decorator(my_sort)"
@sorting_decorator 
def my_sort(a):
    a.sort()
    pass

my_sort()
# sort_sanity(my_sort)
"""

In [2]:
def gen_num(size=20):
    return np.random.choice(20, size=size, replace=False)
    
def is_sorted(a):
    for i in range(len(a)-1):
        if a[i]>a[i+1]:
            return False
    return True

def sort_sanity(func, verbose=True, **kwargs):
    # np.random.seed(1)
    a,b,c = [], [1], np.random.randint(10, size=15)
    for x in [a,b,c]:
        if verbose: 
            print(f"before sort: {x}")
        func(x, **kwargs)
        if verbose:
            print(f"after sort: {x}")
        assert is_sorted(x)
    print("all tests passed...")

# insertion sort (in-place)
idea: given array a, we start by looking at `a[:1]`. It's already sorted, since `a[:1]` contains one element only. Then. look at `a[:2]`. We need to put `a[1]` into the correct position relative to `a[:2]`. In general, at ith iteration, we need to insert `a[i-1]` into the correct position of `a[:i]`, where `a[:i-1]` is already sorted.  

To think about what to do after exiting while loop, think of the case when i>=0 is false. In this case, i == -1, and so we should insert in i+1, not i!

In [20]:
def insert(a):
    # assume a[:-1] is sorted. put the last element a[-1] into the correct position so the entire array a is sorted
    # e.g., a = [1,2,4,5,3]
    # n = len(a)
    # i = len(a)-2
    # while i>=0 and a[i]>a[i+1]:
    #     a[i],a[i+1] = a[i+1], a[i]
    #     i-=1

    # v2: do not swap
    val = a[-1]
    i = len(a)-2
    while i >= 0 and a[i] > val:
        a[i+1] = a[i]
        i-=1
    a[i+1]=val
    
# a=[2,3,7,9,1]
# a=[2,3,7,9,8]
# insert(a)
# print(a)

def insertion_sort(a, ver=1):
    n = len(a)

    def v1():
        # M1: bad method. Use `for` and `break` !
        for i, key in enumerate(a):
            for cur in range(i-1, -1, -1):
                if a[cur]>a[cur+1]:
                    a[cur], a[cur+1] = a[cur+1], a[cur] 
                else:
                    break

    def v2():
        for i, key in enumerate(a):
            print(a)
            for cur in range(i-1, -1, -1):
                if a[cur]>key:
                    a[cur+1] = a[cur]
                else:
                    a[cur+1] = key 
                    break
    
    def v3():
        # good version. Use while
        for i in range(n):
            cur = a[i]
            j = i-1
            while j >= 0 and a[j] > cur:
                # think about the case where cur is the minimum
                a[j+1] = a[j]
                j -= 1
            a[j+1] = cur # if already in the correct position (i.e., while loop doesn't get executed), then it will do a[j+1] = cur = a[j+1]

        # alternative: cleaner version
        # for i, x in enumerate(a):
        #     j = i-1
        #     while j >=0 and a[j] > x:
        #         a[j+1] = a[j]
        #         j-=1
        #     a[j+1] = x 

    def v4():
        # swap every time. Inefficient but accord with explanation
        for i in range(1,n):
            j = i-1
            while j>=0 and a[j]>a[j+1]:
                a[j],a[j+1] = a[j+1], a[j]
                j-=1

    def insertion_sort_rec(a,i):
        # recursion
        # i is the index of the last element
        if i==0: return # one element only; already sorted
        insertion_sort_rec(a, i-1)
        cur=i-1
        while cur>=0 and a[cur]>a[cur+1]:
            a[cur],a[cur+1]=a[cur+1],a[cur]
            cur-=1
    
    eval(f'v{ver}()') if isinstance(ver, int) else eval(f'{ver}()')

# sort_sanity(insertion_sort, 'insertion_sort_rec')
sort_sanity(insertion_sort, True, ver=4)

before sort: []
after sort: []
before sort: [1]
after sort: [1]
before sort: [9 2 0 9 1 9 0 6 0 4 8 4 3 3 8]
after sort: [0 0 0 1 2 3 3 4 4 6 8 8 9 9 9]
all tests passed...


# bubble sort (in-place)
- Bubble Sort is the simplest sorting algorithm that works by repeatedly swapping the adjacent elements if they are in wrong order.
- after a pass, the largest element will be in the correct position
- `n_passes = n-1`
- `n_comparisons = (n-1)-i`
- if there is no swapping in a pass, that means the numbers are already sorted, and so we can terminate immediately

In [29]:
def bubble_sort(a, ver=2):
    n = len(a)
    def v1():
        # no early termination
        for i in range(n-1): # i is the number of passes. At nth iteration, there is only one number, and so it must already be the smallest, and so we do not need to go to nth iteration 
            for j in range(0, (n-1)-i): # -1 is needed, because if not, then when i=0, in the for loop we'll have a[n-1]>a[n], so out of bound. Another explanation: if we have 5 numbers, then we need to do neighbor comparsions 4 times only, so (n-1) needed. 
                if a[j] > a[j+1]: # in wrong order. Need to swap them
                    a[j], a[j+1] = a[j+1], a[j] 
    def v2():
        # early termination using for loop
        # this is based on the previous version. We just add a flag `no_swap` here. Recall that if there is no swap in a pass, that means it's already sorted
        # this is cleaner than using while loop, since this version is based on the original version. 
        for i in range(n-1):
            sorted=True
            for j in range(0, (n-1)-i):
                if a[j] > a[j+1]: # in wrong order. Need to swap them
                    a[j], a[j+1] = a[j+1], a[j] 
                    sorted = False
            if sorted:
                return
    def v3():
        # early termination with while loop
        # cons: need to define variable `sorted` at the beginning first...
        i = 0
        sorted=False
        while i < n-1 and not sorted:
            sorted=True
            for j in range(0, (n-1)-i):
                if a[j] > a[j+1]: # in wrong order. Need to swap them
                    a[j], a[j+1] = a[j+1], a[j] 
                    sorted = False
            i+=1
            
    eval(f'v{ver}()') if isinstance(ver, int) else eval(f'{ver}()')

sort_sanity(bubble_sort, ver=3)

before sort: []
after sort: []
before sort: [1]
after sort: [1]
before sort: [5 0 3 1 4 4 4 0 0 8 4 6 9 3 3]
after sort: [0 0 0 1 3 3 3 4 4 4 4 5 6 8 9]
all tests passed...


# selection sort (in-place)
- In ith iteration, it finds the ith smallest (or largest) elements and put it in ith position. 
- early termination: 

In [77]:
def selection_sort(a, ver):
    n = len(a)
    def v1():
        # find min version
        for i in range(n):
            min_index = i
            for j in range(i+1, n):
                if a[min_index] > a[j]:
                    min_index = j
            a[i], a[min_index] = a[min_index], a[i]
    def v2():
        # find max version
        for i in range(n-1, -1, -1):
            max_index = 0
            for j in range(1, i+1):
                if a[max_index] < a[j]:
                    max_index = j
            a[i], a[max_index] = a[max_index], a[i]
    def v3():
        # early termination + find max + while loop
        """ 
        early termination version of selection sort. 
        this version finds the max element in each iteration.
        """
        n = len(a) # number of elements in the input sequence
        sorted = False
        i = n-1 # start from the last element, which has index n-1. 
        while i>0 and not sorted:
            # print(a)
            max_index = 0 # find the max index, starting from the first element
            sorted = True
            for j in range(1, i+1): 
                if a[max_index] < a[j]: # note: for [1,1,1,1], we still can't early terminate..
                    max_index = j
                else:
                    sorted = False
            a[i], a[max_index] = a[max_index], a[i] # swap elements
            i-=1
    def v4():
        # early termination + find max + for loop
        for i in range(n-1, -1, -1):
            max_index = 0
            sorted = True
            # print(a)
            for j in range(1, i+1):
                if a[j] > a[max_index]:
                    max_index = j
                else:
                    sorted = False
            if sorted:
                return 
            a[i], a[max_index] = a[max_index], a[i]
    def selection_sort_recur(a,i):
        # recursion 
        # i is the index of last elements in a
        if i==0: return 
        max_index=0
        for j in range(1,i+1):
            if a[max_index]<a[j]: max_index=j
        a[i],a[max_index]=a[max_index],a[i]
        selection_sort_recur(a,i-1)
    def selection_sort_abridged(a):
        # this uses numpy's argmin function to make the code shorter
        for i in range(len(a)-1):
            min_index=np.argmin(a[i:])+i
            a[i],a[min_index]=a[min_index],a[i]
        return a

    eval(f'v{ver}()') if isinstance(ver, int) else eval(f'{ver}()')

        
sort_sanity(selection_sort, True, ver=3)


before sort: []
after sort: []
before sort: [1]
after sort: [1]
before sort: [1 5 9 4 5 7 4 4 2 2 1 3 0 1 9]
after sort: [0 1 1 1 2 2 3 4 4 4 5 5 7 9 9]
all tests passed...


# merge sort (in-place)
- we first implement a function called `merge(a,p,q,r)`. Assume that `a[p:q+1]` and `a[q+1:r+1]` are sorted
- notation: a[p..q] <=> a[p:q+1]

In [19]:
def merge(a,low,mid,high):
    i=j=0
    k = low
    left, right = a[low:mid+1], a[mid+1:high+1]
    
    # M1: CLRS
    # while i < len(left) and j < len(right):
    #     if left[i] < right[j]:
    #         a[k] = left[i]
    #         i+=1
    #     else:
    #         a[k] = right[j]
    #         j+=1
    #     k+=1
    # a[k:high+1] = left[i:] if i < len(left) else right[j:]
    
    # M2: advanced
    for k in range(low, high+1):
        if j>=len(right) or (i < len(left) and left[i] < right[j]):
            a[k] = left[i]
            i+=1
        else:
            a[k] = right[j]
            j+=1

def merge_sort(a, low, high):
    if low >= high:
        return 
    mid = (low+high)//2
    merge_sort(a, low, mid)
    merge_sort(a, mid+1, high)
    merge(a, low, mid, high)
    
a=[1,2,3,-1,3,5,190,777]
print(a)
merge_sort(a, 0, len(a)-1)
a

[1, 2, 3, -1, 3, 5, 190, 777]


[-1, 1, 2, 3, 3, 5, 190, 777]

In [158]:
def merge(a,low,mid,high):
    # assume a[low..mid] and a[mid+1..high] are sorted
    # idea: 
        # create two temp lists: left and right
        # create two pointers (i and j) pointing to the beginning of left and right
    
    i=j=0
    left, right = a[low:mid+1], a[mid+1:high+1]
    n_left, n_right = mid + 1 - low, high - mid
    
    # M0: CLRS: don't need to create `n_left` and `n_right`! using len(left) is constant time
    # while i < len(left) and j < len(right):
    #     if left[i] < right[j]:
    #         a[k] = left[i]
    #         i+=1
    #     else:
    #         a[k] = right[j]
    #         j+=1
    #     k+=1
    # a[k:high+1] = left[i:] if i < len(left) else right[j:]

    # M1: most basic way 
    # k=low
    # while i<n_left and j<n_right:
    #     if left[i] < right[j]:
    #         a[k] = left[i]
    #         i+=1
    #     else:
    #         a[k] = right[j]
    #         j+=1  
    #     k+=1
    # while i<n_left:
    #     a[k] = left[i]
    #     i+=1
    #     k+=1
    # while j<n_right:
    #     a[k] = right[j]
    #     j+=1
    #     k+=1

    # M2: advanced way
    # for k in range(low, high+1):
    #     # think: when should we put the element in list array to a?
    #     if j>= n_right or (i<n_left and left[i]<right[j]):
    #         a[k] = left[i]
    #         i+=1
    #     else:
    #         a[k] = right[j]
    #         j+=1

    # M3: sentinel,  i.e., append float('inf') at the end 
    # good thing: don't need to have n_left or n_right!!!
    # left.append(float('inf'))
    # right.append(float('inf'))
    # for k in range(low, high+1):
    #     if left[i]<right[j]:
    #         a[k] = left[i]
    #         i+=1
    #     else:
    #         a[k] = right[j]
    #         j+=1

    
    # M4: pythonic, but slow 
    # it uses list.pop(0)!!
    # k=low
    # while left and right: # neither half is empty
    #     a[k]=left.pop(0) if left[0]<right[0] else right.pop(0) # append and remove the smallest element 
    #     k+=1
    # a[k:high+1] = (left or right) # also add the remainder

    # M5: python and fast!
    # use list.pop() only!
    k = high
    while left and right: # neither half is empty
        a[k]=left.pop() if left[-1]>right[-1] else right.pop() # append and remove the largest element 
        k-=1
    a[low:k+1] = (left or right) # also add the remainder


def merge_sort(a, low, high):
    # print(a[low:high+1])
    if low >= high: # 0 or 1 element
        return 

    mid = (low+high)//2
    merge_sort(a, low, mid)
    merge_sort(a, mid+1, high)
    merge(a, low, mid, high)


# def merge_sort_inplace(a):
#     def merge_sort_inplace_aux(a,low,high):
#         if high<=low: return
#         mid=(high+low)//2
#         merge_sort_inplace_aux(a,low, mid)
#         merge_sort_inplace_aux(a,mid+1, high)
#         merge(a, low, mid, high)
#     merge_sort_inplace_aux(a, 0,len(a)-1)

# a = [9,1,2,7,8,3,4,6,9, -1]
# low=1
# mid=4
# high=8

# a = [1,2,7,8,3,4,6,9]
# low = 0
# mid = 3
# high = 7
# merge(a, low, mid, high)
# print(a)

# a = list(gen_num(5))
a=[1,2,3,-1,3,5,190,777]
print(a)
merge_sort(a, 0, len(a)-1)
a

[1, 2, 3, -1, 3, 5, 190, 777]


[-1, 1, 2, 3, 3, 5, 190, 777]

# quick sort (in-place)

In [ ]:
def partition(a, start, end, random_pivot=True):
    pivot = a[end]
    left = start -1
    for right in range(start, end):
        if a[right] < pivot:
            left += 1
            a[left], a[right] = a[right], a[left]
    a[left+1], a[end] = a[end], a[left+1]

In [23]:
def partition(a, start, end, random_pivot=True):
    # partition is inclusive in terms of start and end. That is, from a[start] to a[end], inclusive. So, to partition the entire array, call partition(a,0,len(a)-1)
    # - if given array portion has two elements only, the for loop will still execute once
    # - if given array portion has one element only, the for loop WON'T execute, and the final swapping is with itself
    # - return the final index of pivot 
    # - Note: when thinking about the algorithm, think of it in its intermediate step!
    # partition into two parts such that `left_part <= pivot < right_part`
    # this is the method from CLRS
    left=start-1
    
    if random_pivot:
        num=np.random.randint(start, end+1)
        a[num], a[end] = a[end], a[num]
    
    pivot=a[end]
    for right in range(start, end): # not end-1!!
        if a[right]<=pivot: # cur should be on the `less than` region => increase i and then swap that element with cur, because after increasing, i's element is larger than pivot 
            left+=1
            a[left],a[right]=a[right],a[left]
            
    # using a[left] will be wrong, since a[left] < pivot, and if we put a[left] to the end, it will be wrong
    a[left+1],a[end]=a[end],a[left+1]
    return left+1 # final index of pivot. The value is relative to the whole a!

    
    # ---------------------- M2: use first element as pivot ---------------------- #
    # partition into two parts such that `left_part < pivot <= right_part`
    # from SEHH2239
    
    # i = start
    # pivot = a[start]
    # for j in range(start+1, end+1):
    #     if a[j] < pivot:
    #         i+=1
    #         a[i], a[j] = a[j], a[i]
    # a[start], a[i] = a[i], a[start]
    # print(a)
    # return i

def quick_sort(a, start, end):
    if start>=end:
        return
    pivot_index = partition(a, start, end)
    quick_sort(a, start, pivot_index-1)
    quick_sort(a, pivot_index+1, end)

# def quick_sort(a):
#     def quick_sort_helper(a,start, end):
#         if start>=end: return 
#         pivot_index=partition(a, start, end)
#         quick_sort_helper(a,start, pivot_index-1) 
#         quick_sort_helper(a,pivot_index+1, end)
#     quick_sort_helper(a, 0, len(a)-1)

# driver code
a=[2, 8, 7, 1, 3, 5, 6, 4]
a=[2, 2, 3, 3, 3, 3, 2, 1]
a=[]
quick_sort(a, 0, len(a)-1)
print(a)

[]


# non-inplace quick sort. Remember that this implementation is NON-INPLACE

In [7]:
def quick_sort(a):
    """non-inplace quick sort. the last number of the input sequence `a` (i.e., a[-1]) is chosen as the pivot

    Args:
        a (sequence): a sequence of numbers

    Returns:
        sequence: the sorted `a`
    """    

    if len(a)<=1: return a
    # if len(a)==0: return [] # also ok

    # a[-1] is the pivot.
    pivot=a[-1]
    return quick_sort([x for x in a[:-1] if x<=pivot]) \
            + [pivot] \
            + quick_sort( [x for x in a[:-1]  if x>pivot])

# driver code
a=[4,3,3,1,4,5]
print(quick_sort(a))


[1, 3, 3, 4, 4, 5]


# counting sort
Counting sort assumes that each of the input elements is an integer in the range 0 to k, for some integer k. It runs in time, so that when , counting sort runs in time. Counting sort first determines, for each input element , the number of elements less than or equal to . It then uses this information to place element directly into its position in the output array. For example, if elements are less than or equal to , then belongs in output position . We must modify this scheme slightly to handle the situation in which several elements have the same value, since we do not want them all to end up in the same position. The COUNTING-SORT procedure on the facing page takes as input an array , the size of this array, and the limit on the nonnegative integer values in . It returns its sorted output in the array and uses an array for temporary working storage. �kW0ŒC �nW1ŒB

In [25]:
def counting_sort(a, high_lim, debug=False):
    # input a = [2,5,3,0,2,3,0,3]
    count = [0] * (high_lim+1)
    res = [-1] * len(a)
    for n in a:
        count[n] += 1 # count[i] = # times i appears in the input array
    for i in range(1, len(count)):
        count[i] += count[i-1] # count[i] = # elements in the input array <= i
                                # eg: count[0] = 2 => there are 2 elements in the array <= 0
        # [2, 2, 4, 7, 7, 8]
        
    # now, to get the correct position of a[i] we just find count[a[i]-1], the -1 is because the index start from 0
    for i, n in enumerate(reversed(a)): # reversed to ensure that the result is stable
        print("----------before insertion----------")
        print(f"{n = }")
        print(f"{count = }")
        print(f"{res = }")
        res[count[n]-1] = n
        count[n] -= 1
        print("----------after insertion----------")
        print(f"{count = }")
        print(f"{res = }")
    return res 

a = [2, 5, 3, 0, 2, 3, 0, 3]
print(a)
counting_sort(a, max(a))

# driver code
a = [2, 5, 3, 0, 2, 3, 0, 3]
counting_sort(a)


[0, 0, 2, 2, 3, 3, 3, 5]

In [ ]:
%time sum(i for i in range(10**8))


CPU times: user 2.58 s, sys: 5.57 ms, total: 2.59 s
Wall time: 2.59 s


4999999950000000